# Building an image retrieval system with deep features


# Fire up GraphLab Create

In [1]:
import graphlab

# Load the CIFAR-10 dataset

We will use a popular benchmark dataset in computer vision called CIFAR-10.  

(We've reduced the data to just 4 categories = {'cat','bird','automobile','dog'}.)

This dataset is already split into a training set and test set. In this simple retrieval example, there is no notion of "testing", so we will only use the training data.

In [2]:
image_train = graphlab.SFrame('image_train_data/')

[INFO] This non-commercial license of GraphLab Create is assigned to p.e.strickland@gmail.com and will expire on January 15, 2017. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-23424 - Server binary: /home/paul/anaconda3/envs/dato/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1455361717.log
[INFO] GraphLab Server Version: 1.8.1


# Computing deep features for our images

The two lines below allow us to compute deep features.  This computation takes a little while, so we have already computed them and saved the results as a column in the data you loaded. 

(Note that if you would like to compute such deep features and have a GPU on your machine, you should use the GPU enabled GraphLab Create, which will be significantly faster for this task.)

In [3]:
#deep_learning_model = graphlab.load_model('http://s3.amazonaws.com/GraphLab-Datasets/deeplearning/imagenet_model_iter45')
#image_train['deep_features'] = deep_learning_model.extract_features(image_train)

PROGRESS: Downloading http://s3.amazonaws.com/GraphLab-Datasets/deeplearning/imagenet_model_iter45/dir_archive.ini to /var/tmp/graphlab-paul/23424/ed2c71e5-2db7-4fde-bf2d-82ee27d8bebd.ini
PROGRESS: Downloading http://s3.amazonaws.com/GraphLab-Datasets/deeplearning/imagenet_model_iter45/objects.bin to /var/tmp/graphlab-paul/23424/4db2e234-c2d5-4fdc-9130-7229716aa802.bin


RuntimeError: Runtime Exception. Unable to load model from http://s3.amazonaws.com/GraphLab-Datasets/deeplearning/imagenet_model_iter45: CUDA Error: out of memory

In [101]:
image_train.head()

id,image,label,deep_features,image_array
24,Height: 32 Width: 32,bird,"[0.242871761322,1.09545373917, 0.0, ...","[73.0, 77.0, 58.0, 71.0,68.0, 50.0, 77.0, 69.0, ..."
33,Height: 32 Width: 32,cat,"[0.525087952614, 0.0,0.0, 0.0, 0.0, 0.0, ...","[7.0, 5.0, 8.0, 7.0, 5.0,8.0, 5.0, 4.0, 6.0, 7.0, ..."
36,Height: 32 Width: 32,cat,"[0.566015958786, 0.0,0.0, 0.0, 0.0, 0.0, ...","[169.0, 122.0, 65.0,131.0, 108.0, 75.0, ..."
70,Height: 32 Width: 32,dog,"[1.12979578972, 0.0, 0.0,0.778194487095, 0.0, ...","[154.0, 179.0, 152.0,159.0, 183.0, 157.0, ..."
90,Height: 32 Width: 32,bird,"[1.71786928177, 0.0, 0.0,0.0, 0.0, 0.0, ...","[216.0, 195.0, 180.0,201.0, 178.0, 160.0, ..."
97,Height: 32 Width: 32,automobile,"[1.57818555832, 0.0, 0.0,0.0, 0.0, 0.0, ...","[33.0, 44.0, 27.0, 29.0,44.0, 31.0, 32.0, 45.0, ..."
107,Height: 32 Width: 32,dog,"[0.0, 0.0,0.220677852631, 0.0, ...","[97.0, 51.0, 31.0, 104.0,58.0, 38.0, 107.0, 61.0, ..."
121,Height: 32 Width: 32,bird,"[0.0, 0.23753464222, 0.0,0.0, 0.0, 0.0, ...","[93.0, 96.0, 88.0, 102.0,106.0, 97.0, 117.0, ..."
136,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 7.5737862587, 0.0, ...","[35.0, 59.0, 53.0, 36.0,56.0, 56.0, 42.0, 62.0, ..."
138,Height: 32 Width: 32,bird,"[0.658935725689, 0.0,0.0, 0.0, 0.0, 0.0, ...","[205.0, 193.0, 195.0,200.0, 187.0, 193.0, ..."


# Train a nearest-neighbors model for retrieving images using deep features

We will now build a simple image retrieval system that finds the nearest neighbors for any image.

In [102]:
knn_model = graphlab.nearest_neighbors.create(image_train,features=['deep_features'],
                                             label='id')

PROGRESS: Starting brute force nearest neighbors model training.


# Use image retrieval model with deep features to find similar images

Let's find similar images to this cat picture.

In [103]:
graphlab.canvas.set_target('ipynb')
cat = image_train[18:19]
cat['image'].show()

In [104]:
knn_model.query(cat)

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.0498753   | 3.32ms       |
PROGRESS: | Done         |         | 100         | 116.927ms    |
PROGRESS: +--------------+---------+-------------+--------------+


query_label,reference_label,distance,rank
0,384,0.0,1
0,6910,36.9403137951,2
0,39777,38.4634888975,3
0,36870,39.7559623119,4
0,41734,39.7866014148,5


We are going to create a simple function to view the nearest neighbors to save typing:

In [105]:
def get_images_from_ids(query_result):
    return image_train.filter_by(query_result['reference_label'],'id')

In [106]:
cat_neighbors = get_images_from_ids(knn_model.query(cat))

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.0498753   | 6.354ms      |
PROGRESS: | Done         |         | 100         | 126.445ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [107]:
cat_neighbors['image'].show()

Very cool results showing similar cats.

## Finding similar images to a car

In [108]:
car = image_train[8:9]
car['image'].show()

In [109]:
get_images_from_ids(knn_model.query(car))['image'].show()

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.0498753   | 8.107ms      |
PROGRESS: | Done         |         | 100         | 121.422ms    |
PROGRESS: +--------------+---------+-------------+--------------+


# Just for fun, let's create a lambda to find and show nearest neighbor images

In [110]:
show_neighbors = lambda i: get_images_from_ids(knn_model.query(image_train[i:i+1]))['image'].show()

In [111]:
show_neighbors(8)

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.0498753   | 3.626ms      |
PROGRESS: | Done         |         | 100         | 132.369ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [112]:
show_neighbors(98)

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.0498753   | 7.509ms      |
PROGRESS: | Done         |         | 100         | 111.73ms     |
PROGRESS: +--------------+---------+-------------+--------------+


## Assignment

In [116]:
image_train["label"].sketch_summary()


+------------------+-------+----------+
|       item       | value | is exact |
+------------------+-------+----------+
|      Length      |  2005 |   Yes    |
| # Missing Values |   0   |   Yes    |
| # unique values  |   4   |    No    |
+------------------+-------+----------+

Most frequent items:
+-------+------------+-----+-----+------+
| value | automobile | cat | dog | bird |
+-------+------------+-----+-----+------+
| count |    509     | 509 | 509 | 478  |
+-------+------------+-----+-----+------+


In [117]:
cat = image_train[image_train["label"] == "cat"]

In [118]:
dog = image_train[image_train["label"] == "dog"]

In [119]:
car = image_train[image_train["label"] == "automobile"] 

In [120]:
bird = image_train[image_train["label"] == "bird"]

In [121]:
cat_model = graphlab.nearest_neighbors.create(cat, features=['deep_features'],
                                             label='id')

PROGRESS: Starting brute force nearest neighbors model training.


In [122]:
dog_model = graphlab.nearest_neighbors.create(dog, features=['deep_features'],
                                             label='id')

PROGRESS: Starting brute force nearest neighbors model training.


In [123]:
car_model = graphlab.nearest_neighbors.create(car, features=['deep_features'],
                                             label='id')

PROGRESS: Starting brute force nearest neighbors model training.


In [124]:
bird_model = graphlab.nearest_neighbors.create(bird, features=['deep_features'],
                                             label='id')

PROGRESS: Starting brute force nearest neighbors model training.


In [128]:
show_cats = lambda i: get_images_from_ids(cat_model.query(image_test[i:i+1]))['image'].show()

Nearest cat image:

In [135]:
cat_nearest = cat_model.query(image_test[0:1])
cat_nearest

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.196464    | 5.919ms      |
PROGRESS: | Done         |         | 100         | 49.929ms     |
PROGRESS: +--------------+---------+-------------+--------------+


query_label,reference_label,distance,rank
0,16289,34.623719208,1
0,45646,36.0068799284,2
0,32139,36.5200813436,3
0,25713,36.7548502521,4
0,331,36.8731228168,5


In [139]:
image_train[image_train["id"] == 16289]["image"].show()

Nearest dog image:

In [140]:
dog_nearest = dog_model.query(image_test[0:1])
dog_nearest

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.196464    | 7.438ms      |
PROGRESS: | Done         |         | 100         | 50.397ms     |
PROGRESS: +--------------+---------+-------------+--------------+


query_label,reference_label,distance,rank
0,16976,37.4642628784,1
0,13387,37.5666832169,2
0,35867,37.6047267079,3
0,44603,37.7065585153,4
0,6094,38.5113254907,5


In [143]:
image_train[image_train["id"] == 16976]["image"].show()

In [141]:
cat_nearest[0:5]["distance"].mean()

36.15573070978294

In [142]:
dog_nearest[0:5]["distance"].mean()

37.77071136184156

## Part 4

In [41]:
image_test = graphlab.SFrame("image_test_data/")

In [42]:
image_test_cat = image_test[image_test["label"] == "cat"]

In [43]:
image_test_dog = image_test[image_test["label"] == "dog"]

In [44]:
image_test_car = image_test[image_test["label"] == "automobile"]

In [45]:
image_test_bird = image_test[image_test["label"] == "bird"]

In [46]:
dog_cat_neighbours = cat_model.query(image_test_dog, k=1)

PROGRESS: Starting blockwise querying.
PROGRESS: max rows per data block: 7668
PROGRESS: number of reference data blocks: 4
PROGRESS: number of query data blocks: 1
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 1000         | 127000  | 24.9509     | 222.804ms    |
PROGRESS: | Done         | 509000  | 100         | 235.616ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [47]:
dog_car_neighbours = car_model.query(image_test_dog, k=1)

PROGRESS: Starting blockwise querying.
PROGRESS: max rows per data block: 7668
PROGRESS: number of reference data blocks: 4
PROGRESS: number of query data blocks: 1
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 1000         | 127000  | 24.9509     | 224.838ms    |
PROGRESS: | Done         | 509000  | 100         | 243.739ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [49]:
dog_bird_neighbours = bird_model.query(image_test_dog, k=1)

PROGRESS: Starting blockwise querying.
PROGRESS: max rows per data block: 7668
PROGRESS: number of reference data blocks: 4
PROGRESS: number of query data blocks: 1
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 1000         | 119000  | 24.8954     | 223.706ms    |
PROGRESS: | Done         | 478000  | 100         | 238.398ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [50]:
dog_dog_neighbours = dog_model.query(image_test_dog, k=1)

PROGRESS: Starting blockwise querying.
PROGRESS: max rows per data block: 7668
PROGRESS: number of reference data blocks: 4
PROGRESS: number of query data blocks: 1
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 1000         | 127000  | 24.9509     | 210.445ms    |
PROGRESS: | Done         | 509000  | 100         | 243.066ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [51]:
dog_dog_neighbours.head()

query_label,reference_label,distance,rank
0,49803,33.4773590373,1
1,5755,32.8458495684,1
2,20715,35.0397073189,1
3,13387,33.9010327697,1
4,12089,37.4849250909,1
5,6094,34.945165344,1
6,3431,39.0957278345,1
7,6184,37.7696131032,1
8,2167,35.1089144603,1
9,7776,43.2422832585,1


In [52]:
dog_distances = graphlab.SFrame({"dog-dog": dog_dog_neighbours["distance"], "dog-cat": dog_cat_neighbours["distance"],
                                "dog-car": dog_car_neighbours["distance"], "dog-bird": dog_bird_neighbours["distance"]})

In [53]:
dog_distances.head()

dog-bird,dog-car,dog-cat,dog-dog
41.7538647304,41.9579761457,36.4196077068,33.4773590373
41.3382958925,46.0021331807,38.8353268874,32.8458495684
38.6157590853,42.9462290692,36.9763410854,35.0397073189
37.0892269954,41.6866060048,34.5750072914,33.9010327697
38.272288694,39.2269664935,34.778824791,37.4849250909
39.1462089236,40.5845117698,35.1171578292,34.945165344
40.523040106,45.1067352961,40.6095830913,39.0957278345
38.1947918393,41.3221140974,39.9036867306,37.7696131032
40.1567131661,41.8244654995,38.0674700168,35.1089144603
45.5597962603,45.4976929401,42.7258732951,43.2422832585


In [63]:
min(dog_distances[0], key=dog_distances[0].get)

'dog-dog'

In [77]:
type(dog_distances[0])

dict

In [76]:
dog_distances[0]

{'dog-bird': 41.753864730351246,
 'dog-car': 41.957976145712024,
 'dog-cat': 36.41960770675437,
 'dog-dog': 33.47735903726335}

In [ ]:
dog_distances[0].get

In [78]:
def is_dog_correct(row):
    if min(row, key=row.get) == "dog-dog":
        return 1
    else: 
        return 0    

In [81]:
dog_distances.apply(is_dog_correct).sum()

678

In [144]:
dog_distances.num_rows

<bound method SFrame.num_rows of Columns:
	dog-bird	float
	dog-car	float
	dog-cat	float
	dog-dog	float

Rows: 1000

Data:
+---------------+---------------+---------------+---------------+
|    dog-bird   |    dog-car    |    dog-cat    |    dog-dog    |
+---------------+---------------+---------------+---------------+
| 41.7538647304 | 41.9579761457 | 36.4196077068 | 33.4773590373 |
| 41.3382958925 | 46.0021331807 | 38.8353268874 | 32.8458495684 |
| 38.6157590853 | 42.9462290692 | 36.9763410854 | 35.0397073189 |
| 37.0892269954 | 41.6866060048 | 34.5750072914 | 33.9010327697 |
|  38.272288694 | 39.2269664935 |  34.778824791 | 37.4849250909 |
| 39.1462089236 | 40.5845117698 | 35.1171578292 |  34.945165344 |
|  40.523040106 | 45.1067352961 | 40.6095830913 | 39.0957278345 |
| 38.1947918393 | 41.3221140974 | 39.9036867306 | 37.7696131032 |
| 40.1567131661 | 41.8244654995 | 38.0674700168 | 35.1089144603 |
| 45.5597962603 | 45.4976929401 | 42.7258732951 | 43.2422832585 |
+---------------+---

In [91]:
def is_cat_correct(row):
    if min(row, key=row.get) == "cat-cat":
        return 1
    else: 
        return 0    

In [85]:
cat_cat_neighbours = cat_model.query(image_test_cat, k=1)

PROGRESS: Starting blockwise querying.
PROGRESS: max rows per data block: 7668
PROGRESS: number of reference data blocks: 4
PROGRESS: number of query data blocks: 1
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 1000         | 127000  | 24.9509     | 217.745ms    |
PROGRESS: | Done         | 509000  | 100         | 246.815ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [86]:
cat_dog_neighbours = dog_model.query(image_test_cat, k=1)

PROGRESS: Starting blockwise querying.
PROGRESS: max rows per data block: 7668
PROGRESS: number of reference data blocks: 4
PROGRESS: number of query data blocks: 1
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 1000         | 127000  | 24.9509     | 213.448ms    |
PROGRESS: | Done         | 509000  | 100         | 238.609ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [87]:
cat_car_neighbours = car_model.query(image_test_cat, k=1)

PROGRESS: Starting blockwise querying.
PROGRESS: max rows per data block: 7668
PROGRESS: number of reference data blocks: 4
PROGRESS: number of query data blocks: 1
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 1000         | 128000  | 25.1473     | 227.037ms    |
PROGRESS: | Done         | 509000  | 100         | 246.725ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [88]:
cat_bird_neighbours = bird_model.query(image_test_cat, k=1)

PROGRESS: Starting blockwise querying.
PROGRESS: max rows per data block: 7668
PROGRESS: number of reference data blocks: 4
PROGRESS: number of query data blocks: 1
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 1000         | 120000  | 25.1046     | 209.398ms    |
PROGRESS: | Done         | 478000  | 100         | 240.065ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [89]:
cat_distances = graphlab.SFrame({"cat-cat": cat_cat_neighbours["distance"], "cat-dog": cat_dog_neighbours["distance"],
                                "cat-car": cat_car_neighbours["distance"], "cat-bird": cat_bird_neighbours["distance"]})

In [92]:
cat_distances.apply(is_cat_correct).sum()

548